### Natural Language Processing




---





You have to install the urduhack on your system or colab by using given command.



In [1]:
#Pre-reqs

In [1]:
import csv
import urduhack
from urduhack.tokenization import sentence_tokenizer

#### Reading Training File

In [2]:
with open('urdu-train.txt', 'rt', encoding="utf-8") as f:
    reader = csv.reader(f)
    print(reader)
    train_data = list(reader)
f.close()

#### Reading Labeled File

In [3]:
with open('urdu-test.txt', 'rt', encoding="utf-8") as f:
    reader = csv.reader(f)
    test_data = list(reader)
f.close()

#### Accuracy

##### For Single Paragraph

In [4]:
def acc(test_text,label_text):
    
    correct=0
    for i in label_text:
        for j in test_text:
            lb=i.split(' ')
            ts=j.split(' ')
            if(len(lb)>2 and len(ts)>2):
                if(lb[-1]==ts[-1] and lb[-2]==ts[-2] and lb[-3]==ts[-3]):
                    correct=correct+1
            else:
                return('Error')
    accuracy=(float(correct)/float(len(label_text)))*100
      # print('Accuracy\t=\t'+str(accuracy)+'%')
    return accuracy

##### For Multiple Paragraph

In [5]:
def Accuracy(train,test):
    total=0
    e=0
    for i in range(len(test)):
        train_text = sentence_tokenizer(train[i][0])
        label_text = test[i][0].split('۔')
        tmp=acc(train_text,label_text)
        if(tmp!='Error'):
            total=total+tmp
        else:
            e=e+1

    accuracy=(float(total)/float(len(test)-e))
    print('Accuracy\t=\t'+str(accuracy)+'%')



In [6]:
test = train_data[0][0]
label = test_data[0][0]

#### URDUhack

In [7]:
test_text = sentence_tokenizer(test)

Code here..............................

In [8]:
#For training data.

def split_using_delimeter(_str, separator):
    """Replace end of sentence with separator"""
    if not _str:
        return []
    max_p = chr(ord(max(_str)) + 1)
    return _str.replace(separator, separator + max_p).split(max_p)

def get_endwords_and_conjunctions(text):
    endwords = ['رہے','کیجیے', 'کیجئے', 'گئیں','دیگی','کئے','کرنا','ہوں','ملینگے','جائے','لگا','کہا','کیں','لگی','تھیں', 'تھا', 'دیگی', 'رکھی', 'تھی', 'رکھا', 'ہوی', 'تھے', 'چاہیے','ملیںگی', 'گا', 'کرنا', 'گیا', 'دیا', 'ہونگے', 'گی', 'دیگا', 'ہیں', 'لیں', 'ہے',
                    ]
    conjunctions = ['یا','اگرچہ','یعنی','گویا','جبکہ','جب کہ','جن','تو','اور','جسے', 'تاہم', 'جس', 'بلکہ', 'مگر', 'کیونکہ', 'پر',
                    ]
    
    sentences = split_using_delimeter(text, '۔')
    
    for _sen in sentences:
        _sen = _sen.replace('۔', '')
        _sen = _sen.replace('؟', '')
        
        _sen = _sen.split() 
        if _sen:     
            endwords.append(_sen[-1])   #Picking the words before delimiter 
            

        for index, word in enumerate(_sen):
            if word in endwords and len(_sen)-1!=index:   #If word is an endword, length of sentence is not null and word+1 is not an endword, then word+1 is a conjunction
                if _sen[index+1] not in endwords:
                    conjunctions.append(_sen[index+1])
                    
    return set(endwords), set(conjunctions)

endwords, conjunctions = get_endwords_and_conjunctions(label)

# print(endwords)
# print(conjunctions)

def lets_segment(text):
    sentences = split_using_delimeter(text, '۔')

    for each in sentences:
        each = each.split()

    allsentences = []
    newsen= []
    for index, word in enumerate(each):
        if len(each)-1!=index:
            if (word not in endwords) or (word in endwords and (each[index+1] in conjunctions or each[index+1] in endwords)):   #Append words if sentence not finished
                newsen.append(word)
            else:
                newsen.append(word)          #Endword of the sentence reached, so append
                newsen = ' '.join(newsen)    #Stringify
                allsentences.append(newsen)  #Add to list of sentences
                newsen = []
                
    return allsentences


train_data = lets_segment(test)

# seg = '۔ '.join(train_data)
# print(seg)

#### Label

In [9]:
label_text=label.split('۔')

#### Results

In [10]:
#Urduhack accuracy
acc(test_text,label_text)

74.07407407407408

In [11]:
#Custom algorithm accuracy
acc(train_data, label_text)

81.48148148148148